In [3]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import os
import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define paths
root_path = '/content/drive/My Drive'
template_directory = os.path.join(root_path, 'constellations')
image_directory = os.path.join(root_path, 'images')

def dist(p1, p2):
    return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def getAngle(p0, p1, p2):
    return math.acos((dist(p0, p1) ** 2 + dist(p0, p2) ** 2 - dist(p1, p2) ** 2) / (2 * (dist(p0, p1) ** 2) * (dist(p0, p2) ** 2)))

def getNormalisedCoordinates(x, y, brightest_index, second_brightest_index, lines=[]):
    x = copy.deepcopy(x)
    y = copy.deepcopy(y)

    lines = np.array(lines)
    for line in lines:
        for x1, y1, x2, y2 in line:
            x1 -= x[brightest_index]
            y1 -= y[brightest_index]
            x2 -= x[brightest_index]
            y2 -= y[brightest_index]
            line[0][0] = x1
            line[0][1] = y1
            line[0][2] = x2
            line[0][3] = y2

    for i in range(len(x)):
        x[len(x) - i - 1] -= x[brightest_index]
        y[len(x) - i - 1] -= y[brightest_index]

    distance_brightest = math.sqrt((x[brightest_index] - x[second_brightest_index]) ** 2 + (y[brightest_index] - y[second_brightest_index]) ** 2)

    for i in range(len(x)):
        x[i] = x[i] / distance_brightest
        y[i] = y[i] / distance_brightest
    lines = lines / distance_brightest

    p0 = (x[brightest_index], y[brightest_index])
    p1 = (x[second_brightest_index], y[second_brightest_index])
    p2 = (1, 0)
    theta = getAngle(p0, p1, p2)
    if round(x[second_brightest_index] * math.cos(theta) - y[second_brightest_index] * math.sin(theta), 2) != float(1) or round(x[second_brightest_index] * math.sin(theta) + y[second_brightest_index] * math.cos(theta), 2) != float(0):
        theta = -theta

    for i in range(1, len(x)):
        x_new = x[i] * math.cos(theta) - y[i] * math.sin(theta)
        y_new = x[i] * math.sin(theta) + y[i] * math.cos(theta)
        x[i], y[i] = round(x_new, 2), round(y_new, 2)

    for line in lines:
        for x1, y1, x2, y2 in line:
            x1_new = x1 * math.cos(theta) - y1 * math.sin(theta)
            y1_new = x1 * math.sin(theta) + y1 * math.cos(theta)
            x2_new = x2 * math.cos(theta) - y2 * math.sin(theta)
            y2_new = x2 * math.sin(theta) + y2 * math.cos(theta)
            line[0][0] = x1_new
            line[0][1] = y1_new
            line[0][2] = x2_new
            line[0][3] = y2_new

    return np.array(x), np.array(y), lines

def iterateArea(contours, lines=[], iterate=False):
    lines = np.array(lines)

    coordinates = {}
    for i in range(len(contours)):
        cnt = contours[i]
        M = cv2.moments(cnt)
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        area = cv2.contourArea(cnt)
        if area in coordinates:
            coordinates[area + 0.0001] = (cx, cy)
        else:
            coordinates[area] = (cx, cy)

    sorted_area = []
    for i in reversed(sorted(coordinates.keys())):
        sorted_area.append(i)

    x = []
    y = []
    for area in sorted_area:
        x.append(coordinates[area][0])
        y.append(coordinates[area][1])
    if len(sorted_area) < 3:
        return [], [], []

    threshold = min(150, sorted_area[2])
    count = 0
    for area in sorted_area:
        if area >= threshold:
            count += 1

    coordinates_list = []
    if not iterate:
        return getNormalisedCoordinates(x, y, 0, 1, lines)
    else:
        for i in range(count):
            for j in range(i + 1, count):
                x_new, y_new, _ = getNormalisedCoordinates(x[i:], y[i:], i, j, lines)
                coordinates_list.append((x_new, y_new))

        return coordinates_list
def findEdges(image, thresh1, thresh2):
    out = cv2.Canny(np.array(image), thresh1, thresh2)
    return out

def invertImage(image):
    return cv2.bitwise_not(image)

def getGrayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def applyMedian(image, size):
    return cv2.medianBlur(image, size)

def binariseImage(img, thresholds):
    output_thresh = []
    for threshold in thresholds:
        ret, thresh = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
        output_thresh.append(thresh)
    return output_thresh

def getGreenChannel(img):
    image_copy_green = img.copy()
    image_copy_green[:, :, 0] = 0
    image_copy_green[:, :, 2] = 0
    return image_copy_green

def getBlueChannel(img):
    image_copy_blue = img.copy()
    image_copy_blue[:, :, 1] = 0
    image_copy_blue[:, :, 2] = 0
    return image_copy_blue

def getRedChannel(img):
    image_copy = img.copy()
    image_copy[:, :, 1] = 0
    image_copy[:, :, 0] = 0
    return image_copy

def makeTemplates(template_directory):
    templates_coordinates = {}

    for filename in os.listdir(template_directory):
        img = cv2.imread(os.path.join(template_directory, filename))
        red_channel = getRedChannel(img)
        thresh = binariseImage(red_channel, [165.75, 191.25])

        blue_channel = getBlueChannel(img)
        thresh2 = binariseImage(red_channel, [125.75, 255])
        letters = thresh2[0] - thresh2[1]
        new_blue = blue_channel + letters

        for i in range(len(new_blue)):
            for j in range(len(new_blue[i])):
                if new_blue[i][j][2] != 0:
                    new_blue[i][j][1] = 0
                    new_blue[i][j][0] = 0
                    new_blue[i][j][2] = 0

        final = thresh[0] - thresh[1]
        stars = applyMedian(final, 3)
        lines = applyMedian(new_blue, 3)
        stars_grey = getGrayscale(stars)
        lines_grey = getGrayscale(lines)
        final_stars = binariseImage(stars_grey, [20])
        final_lines = binariseImage(lines_grey, [5])
        final_stars_inverted = invertImage(final_stars[0])
        final_lines_inverted = invertImage(final_lines[0])
        final_lines = applyMedian(final_lines[0], 3)

        edged = findEdges(final_stars_inverted, 30, 200)

        rho = 1
        theta = np.pi / 180
        threshold = 10
        min_line_length = 2
        max_line_gap = 3
        line_image = np.copy(img) * 0

        drawn_lines = cv2.HoughLinesP(final_lines, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

        if drawn_lines is not None:
            for line in drawn_lines:
                for x1, y1, x2, y2 in line:
                    cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 5)

        edge_copy = edged.copy()
        contours, hierarchy = cv2.findContours(edge_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        final_contours = [contour for contour in contours if cv2.contourArea(contour) != 0]

        x, y, normalised_lines = iterateArea(final_contours, drawn_lines)
        if len(x) == 0 or len(y) == 0:
            continue

        templates_coordinates[filename.split('.')[0]] = (x, y, len(x), normalised_lines)

    return templates_coordinates

def featureExtraction(template):
    x, y, num_stars, lines = template
    feature_vector = np.zeros(num_stars * 2 + len(lines.flatten()))
    feature_vector[:num_stars] = x
    feature_vector[num_stars:num_stars*2] = y
    feature_vector[num_stars*2:] = lines.flatten()
    return feature_vector

def prepare_data(templates_coordinates):
    X = []
    y = []
    labels = list(templates_coordinates.keys())
    label_to_index = {label: index for index, label in enumerate(labels)}

    max_length = max(len(featureExtraction(template)) for template in templates_coordinates.values())

    for label, template in templates_coordinates.items():
        feature_vector = featureExtraction(template)
        padded_feature_vector = np.pad(feature_vector, (0, max_length - len(feature_vector)), 'constant')
        X.append(padded_feature_vector)
        y.append(label_to_index[label])

    X = np.array(X)
    y = np.array(y)
    return X, y, labels

# Prepare data
templates_coordinates = makeTemplates(template_directory)
X, y, labels = prepare_data(templates_coordinates)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Evaluate the classifier
accuracy = knn.score(X_test, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

def classify_image(image_path):
    img = cv2.imread(image_path)
    red_channel = getRedChannel(img)
    thresh = binariseImage(red_channel, [165.75, 191.25])

    blue_channel = getBlueChannel(img)
    thresh2 = binariseImage(red_channel, [125.75, 255])
    letters = thresh2[0] - thresh2[1]
    new_blue = blue_channel + letters

    for i in range(len(new_blue)):
        for j in range(len(new_blue[i])):
            if new_blue[i][j][2] != 0:
                new_blue[i][j][1] = 0
                new_blue[i][j][0] = 0
                new_blue[i][j][2] = 0

    final = thresh[0] - thresh[1]
    stars = applyMedian(final, 3)
    lines = applyMedian(new_blue, 3)
    stars_grey = getGrayscale(stars)
    lines_grey = getGrayscale(lines)
    final_stars = binariseImage(stars_grey, [20])
    final_lines = binariseImage(lines_grey, [5])
    final_stars_inverted = invertImage(final_stars[0])
    final_lines_inverted = invertImage(final_lines[0])
    final_lines = applyMedian(final_lines[0], 3)

    edged = findEdges(final_stars_inverted, 30, 200)

    rho = 1
    theta = np.pi / 180
    threshold = 10
    min_line_length = 2
    max_line_gap = 3
    line_image = np.copy(img) * 0

    drawn_lines = cv2.HoughLinesP(final_lines, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

    if drawn_lines is not None:
        for line in drawn_lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 5)

    edge_copy = edged.copy()
    contours, hierarchy = cv2.findContours(edge_copy, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    final_contours = [contour for contour in contours if cv2.contourArea(contour) != 0]

    coordinates_list = iterateArea(final_contours, drawn_lines, iterate=True)

    # Debugging output
    print(f"Coordinates list for {image_path}: {coordinates_list}")

    if not coordinates_list:
        print(f"No valid coordinates found for image {image_path}")
        return

    for item in coordinates_list:
        if len(item) != 2:
            print(f"Skipping invalid item in coordinates_list: {item}")
            continue
        x, y = item
        if len(x) == 0 or len(y) == 0:
            continue
        feature_vector = np.zeros(len(X[0]))
        num_stars = len(x)
        feature_vector[:num_stars] = x
        feature_vector[num_stars:num_stars*2] = y

        expected_length = len(X[0])
        if drawn_lines is not None:
            flattened_lines = drawn_lines.flatten()
        else:
            flattened_lines = np.zeros(expected_length - num_stars*2)

        feature_vector[num_stars*2:num_stars*2 + len(flattened_lines)] = flattened_lines

        feature_vector = feature_vector[:expected_length]

        prediction = knn.predict([feature_vector])
        predicted_label = labels[prediction[0]]
        print(f"The image {os.path.basename(image_path)} is classified as: {predicted_label}")

# Classify all images in the image directory
def classify_all_images(image_directory):
    for filename in os.listdir(image_directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add more extensions if needed
            image_path = os.path.join(image_directory, filename)
            classify_image(image_path)

# Classify all images
classify_all_images(image_directory)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model accuracy: 0.00%
Coordinates list for /content/drive/My Drive/images/PHOTO-2024-06-22-01-55-17.jpg: ([], [], [])
Skipping invalid item in coordinates_list: []
Skipping invalid item in coordinates_list: []
Skipping invalid item in coordinates_list: []
Coordinates list for /content/drive/My Drive/images/star_test.png: ([], [], [])
Skipping invalid item in coordinates_list: []
Skipping invalid item in coordinates_list: []
Skipping invalid item in coordinates_list: []
Coordinates list for /content/drive/My Drive/images/astrobin_1.jpg: [(array([ 0.  ,  1.  ,  1.16,  1.16,  2.27,  2.54,  1.91,  1.21,  2.35,
        1.61,  1.31,  3.15,  2.1 ,  2.11,  2.06,  3.55,  2.35,  0.78,
        3.64,  2.22,  0.75,  2.56,  2.61,  1.12,  0.99,  1.24,  3.29,
        3.76,  2.1 ,  3.51,  1.11, -3.08,  1.06,  1.97]), array([ 0.  ,  0.  , -0.07, -0.55, -0.28, -0.84, -0.41,  0.